# LoRA implementation

## Set up LoRA

In [1]:
from transformers import GPT2Model, GPT2Tokenizer, GPT2LMHeadModel

model = GPT2Model.from_pretrained('gpt2')
model

/opt/homebrew/Caskroom/miniforge/base/envs/nn-z2h/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2SdpaAttention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

In [2]:
model.h[0]

GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2SdpaAttention(
    (c_attn): Conv1D()
    (c_proj): Conv1D()
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D()
    (c_proj): Conv1D()
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [3]:
model.h[0].attn.c_attn.weight.shape

torch.Size([768, 2304])

Let's do the low rank matrices as `B` (2304 by 4) and `A` (4 by 768).

In [4]:
import torch
import torch.nn as nn

In [5]:
class LoRALayer(nn.Module):
    def __init__(self, in_features, out_features, rank=4):
        super().__init__()
        self.lora_A = nn.Parameter(nn.init.normal_(torch.zeros(rank, in_features)))
        self.lora_B = nn.Parameter(torch.zeros(out_features, rank))
        self.rank = rank
        self.scaling = 1.0

    def forward(self, x):
        return (x@self.lora_A.T@self.lora_B.T) * self.scaling/self.rank


In [6]:
lora_layers = nn.ModuleList([LoRALayer(768, 2304) for _ in range(12)])

In [7]:
import types

In [8]:
for i, block in enumerate(model.h):
    original_forward = block.attn.c_attn.forward

    def new_forward(self, x):
        x = original_forward(x) + lora_layers[i](x)
        return x
    
    block.attn.c_attn.forward = types.MethodType(new_forward, block.attn.c_attn)

In [9]:
# Freeze all parameters of the original model
for param in model.parameters():
    param.requires_grad = False

# Unfreeze LoRA parameters
for layer in lora_layers:
    for param in layer.parameters():
        param.requires_grad = True

In [10]:
optimizer = torch.optim.AdamW(lora_layers.parameters(), lr=1e-3)

In [11]:
model

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2SdpaAttention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

## Load data

In [12]:
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm

In [13]:
class ShakespeareDataset(Dataset):
    def __init__(self, file_path, tokenizer, seq_length=128):
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        
        self.tokenizer = tokenizer
        self.seq_length = seq_length
        self.tokens = tokenizer.encode(text)
        
    def __len__(self):
        return len(self.tokens) - self.seq_length

    def __getitem__(self, idx):
        chunk = self.tokens[idx:idx+self.seq_length+1]
        x = torch.tensor(chunk[:-1], dtype=torch.long)
        y = torch.tensor(chunk[1:], dtype=torch.long)
        return x, y

In [14]:
def create_dataloaders(dataset, batch_size, train_split=0.9):
    train_size = int(train_split * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader

In [15]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

/opt/homebrew/Caskroom/miniforge/base/envs/nn-z2h/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [16]:
dataset = ShakespeareDataset('tiny-shakespeare.txt', tokenizer, seq_length=128)
train_loader, val_loader = create_dataloaders(dataset, batch_size=32)

Token indices sequence length is longer than the specified maximum sequence length for this model (338025 > 1024). Running this sequence through the model will result in indexing errors


## Train LoRA

In [17]:
import torch
import torch.nn as nn
import math

In [18]:
class LoRALayer(nn.Module):
    def __init__(self, in_features, out_features, rank=4, alpha=1):
        super().__init__()
        self.lora_A = nn.Parameter(torch.zeros(rank, in_features))
        self.lora_B = nn.Parameter(torch.zeros(out_features, rank))
        self.rank = rank
        self.scaling = alpha / rank

        # Initialize LoRA parameters
        nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))
        nn.init.zeros_(self.lora_B)

    def forward(self, x):
        return (x @ self.lora_A.T @ self.lora_B.T) * self.scaling

In [19]:
class LoRAWrapper(nn.Module):
    def __init__(self, model, rank=4, alpha=1):
        super().__init__()
        self.model = model
        self.lora_layers = nn.ModuleList([
            LoRALayer(768, 2304, rank, alpha) for _ in range(len(model.transformer.h))
        ])
        
        # Freeze original model parameters
        for param in self.model.parameters():
            param.requires_grad = False
        
        # Enable grad for LoRA parameters
        for layer in self.lora_layers:
            for param in layer.parameters():
                param.requires_grad = True

    def forward(self, *args, **kwargs):
        # Store original forward methods
        original_forwards = [block.attn.c_attn.forward for block in self.model.transformer.h]
        lora_layers = self.lora_layers

        # Replace forward methods with LoRA-augmented versions
        for i, block in enumerate(self.model.transformer.h):
            def new_forward(self, x, i=i):
                return original_forwards[i](x) + lora_layers[i](x)
            block.attn.c_attn.forward = new_forward.__get__(block.attn.c_attn, type(block.attn.c_attn))

        # Call the model with LoRA
        output = self.model(*args, **kwargs)

        # Restore original forward methods
        for i, block in enumerate(self.model.transformer.h):
            block.attn.c_attn.forward = original_forwards[i]

        return output

    def get_lora_parameters(self):
        return self.lora_layers.parameters()


In [20]:
from tqdm import tqdm

In [21]:
def train_lora(model, train_loader, val_loader, optimizer, scheduler, num_epochs, device):
    model.to(device)
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs, labels=targets)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                inputs, targets = batch
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs, labels=targets)
                val_loss += outputs.loss.item()
        
        avg_val_loss = val_loss / len(val_loader)
        print(f"Validation Loss: {avg_val_loss:.4f}")
        

In [22]:
batch_size = 128
seq_length = 16
num_epochs = 1
learning_rate = 5e-5
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [23]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [24]:
lora_model = LoRAWrapper(model, rank=4, alpha=1)

In [25]:
optimizer = torch.optim.AdamW(lora_model.get_lora_parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

In [26]:
train_lora(lora_model, train_loader, val_loader, optimizer, scheduler, num_epochs, device)

Epoch 1/1:   1%|          | 77/9504 [04:45<9:43:28,  3.71s/it] 


KeyboardInterrupt: 